# Gamut Mapping - Compression Functions 01

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.optimize import fsolve

COLOUR_STYLE = colour.plotting.colour_style()
COLOUR_STYLE.update({
    'figure.figsize': (11, 11),
    'legend.framealpha':
    colour.plotting.COLOUR_STYLE_CONSTANTS.opacity.low
})

plt.style.use(COLOUR_STYLE)

plt.style.use('dark_background')

colour.utilities.describe_environment()

colour.utilities.filter_warnings(*[True] * 4);

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-132-g8ed24e8e                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Compression & Utility Functions

In [3]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, (a + b * np.tanh((x - a) / b)), x)


def simple_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + (-1 / ((x - a) / b + 1) + 1) * b, x)


def derivative(x, func, d=1e-7):
    return (func(x + d) - func(x - d)) / (2 * d)


In [4]:
colour.plotting.plot_multi_functions(
    {
        'tanh': tanh_compression_function,
        'd/dx(tanh)': lambda x: derivative(x, tanh_compression_function),
        'simple': simple_compression_function,
        'd/dx(simple)': lambda x: derivative(x, simple_compression_function),
    },
    **{
        'samples':
        np.linspace(0, 2, 1000),
        'plot_kwargs': [
            {
                'c': 'r',
                'linestyle': 'dashdot'
            },
            {
                'c': 'r'
            },
            {
                'c': 'g',
                'linestyle': 'dashdot'
            },
            {
                'c': 'g'
            },
        ]
    },
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Solving Intersection with Y = 1

In [5]:
X, A, I = 1.2, 0.8, 1

figure, axes = colour.plotting.plot_multi_functions(
    {
        'tanh': lambda x: tanh_compression_function(
            x, A, fsolve(lambda x: tanh_compression_function(X, A, x) - I, 0.5)),
        'simple': lambda x: simple_compression_function(
            x, A, fsolve(lambda x: simple_compression_function(X, A, x) - I, 0.5)),
    },
    **{
        'standalone': False,
        'samples':
        np.linspace(0, 2, 1000),
        'plot_kwargs': [
            {
                'c': 'r',
            },
            {
                'c': 'g',
            },
        ]
    },
)

axes.scatter(X, I, c='b', s=50, zorder=4);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …